### Tutorial on Logistic Regression algorithm

#### For more information please visit:
##### https://spark.apache.org/docs/2.1.0/ml-classification-regression.html#logistic-regression

In [2]:
# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline

In [3]:
# Create a spark session
spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()

In [4]:
# Get the libsvm sample data
data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [7]:
# Build a Logistic Regression classifier. For more information about the arguments, refer the website
lr = LogisticRegression()

lr_model = lr.fit(data)

In [9]:
# Print the predictions
train_summary = lr_model.summary
train_summary.predictions.show()


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[39.9727764450750...|[1.0,4.3655982185...|       0.0|
|  1.0|(692,[158,159,160...|[-35.662380562160...|[3.25105944320044...|       1.0|
|  1.0|(692,[124,125,126...|[-39.336799621156...|[8.24603148700906...|       1.0|
|  1.0|(692,[152,153,154...|[-28.219286248176...|[5.55289803944932...|       1.0|
|  1.0|(692,[151,152,153...|[-28.142070329444...|[5.99865861146384...|       1.0|
|  0.0|(692,[129,130,131...|[37.8748140555402...|[1.0,3.5577649524...|       0.0|
|  1.0|(692,[158,159,160...|[-36.610101257122...|[1.26018713626709...|       1.0|
|  1.0|(692,[99,100,101,...|[-29.504314986871...|[1.53616833535573...|       1.0|
|  0.0|(692,[154,155,156...|[13.6899025280154...|[0.99999886616363...|       0.0|
|  0.0|(692,[127

In [10]:
# Evaluate on the data
evaluation = lr_model.evaluate(data)
prob = evaluation.predictions.select('probability')
prob.head(2)

[Row(probability=DenseVector([1.0, 0.0])),
 Row(probability=DenseVector([0.0, 1.0]))]

In [11]:
# Use the evaluator class and calculate the accuracy
predictionAndLabels = evaluation.predictions.select(['prediction', 'label'])

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

acc = evaluator.evaluate(predictionAndLabels)
print(acc)

1.0


### Titanic dataset
#### Based on the passenger features, predict whether they survived or not

In [12]:
data = spark.read.csv("titanic.csv", inferSchema=True, header=True)

In [13]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [14]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [15]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [16]:
mycols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])
final_data = mycols.na.drop()

In [17]:
final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

In [18]:
# Working with the features, convert strings to one hot encoded features
gender_idx = StringIndexer(inputCol='Sex',outputCol='Sex_New')
gender_vec = OneHotEncoder(inputCol='Sex_New',outputCol='Sex_Vec')

embarked_idx = StringIndexer(inputCol='Embarked',outputCol='Embarked_New')
embarked_vec = OneHotEncoder(inputCol='Embarked_New',outputCol='Embarked_Vec')

In [19]:
# Create a feature vector
assembler = VectorAssembler(inputCols=['Survived',
 'Pclass',
 'Sex_Vec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_Vec'], outputCol='features')

In [20]:
# Create a logistic regression object
lr = LogisticRegression(featuresCol='features', labelCol='Survived')

In [21]:
# Example to use pipeline
# This is used to execute multiple stages of feature engineering and fitting the model
pipeline = Pipeline(stages=[gender_idx, gender_vec, embarked_idx ,embarked_vec, assembler, lr])

In [24]:
# Split the data into train and test
train_data, test_data = final_data.randomSplit([0.7,0.3])
train_data.show()

+--------+------+----+----+-----+-----+--------+--------+
|Survived|Pclass| Sex| Age|SibSp|Parch|    Fare|Embarked|
+--------+------+----+----+-----+-----+--------+--------+
|       0|     1|male|18.0|    1|    0|   108.9|       C|
|       0|     1|male|19.0|    3|    2|   263.0|       S|
|       0|     1|male|22.0|    0|    0|135.6333|       C|
|       0|     1|male|24.0|    0|    0|    79.2|       C|
|       0|     1|male|24.0|    0|    1|247.5208|       C|
|       0|     1|male|27.0|    0|    2|   211.5|       C|
|       0|     1|male|28.0|    0|    0|    47.1|       S|
|       0|     1|male|28.0|    1|    0| 82.1708|       C|
|       0|     1|male|29.0|    0|    0|    30.0|       S|
|       0|     1|male|31.0|    0|    0| 50.4958|       S|
|       0|     1|male|36.0|    1|    0|   78.85|       S|
|       0|     1|male|37.0|    0|    1|    29.7|       C|
|       0|     1|male|37.0|    1|    0|    53.1|       S|
|       0|     1|male|38.0|    0|    0|     0.0|       S|
|       0|    

In [27]:
# Fit the pipeline to the training data
lr_model = pipeline.fit(train_data)

In [28]:
# Predict on the test data
test_results = lr_model.transform(test_data)

In [29]:
test_results.select(['Survived','prediction']).show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [32]:
# Evaluate using the Area under the curve metric
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

my_eval.evaluate(test_results)

1.0

### Customer Churn Dataset
#### Based on the customer data, predict whether they will churn or not

In [33]:
data = spark.read.csv("customer_churn.csv", inferSchema=True, header=True)

In [34]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [35]:
data.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [36]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [38]:
# Create a vector assembler
assembler = VectorAssembler(inputCols= ['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol = 'features')

In [39]:
output = assembler.transform(data)
final_output = output.select(['features','churn'])
final_output.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [41]:
# Split the data into train and test
train_data, test_data = final_output.randomSplit([0.7, 0.3])

In [42]:
# Create a logistic regression classifier
lr = LogisticRegression(labelCol='churn')

In [44]:
# Fit the model to the train data
model = lr.fit(train_data)

In [45]:
# Predict on the test data
test_res = model.transform(test_data)

In [46]:
test_res.select(['churn', 'prediction']).show()

+-----+----------+
|churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [49]:
# Perform evaluation on the test data
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

auc = churn_eval.evaluate(test_res)
print("Area under the curve is: ", auc)

Area under the curve is:  0.6940387968613776


In [56]:
### Unlabeled data: Predict the customer churn on new unlabeled data based on the model
unlabeled_data = spark.read.csv("new_customers.csv", inferSchema=True, header=True)

In [57]:
unlabeled_data.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [61]:
# Vectorize the features
final_unlab = assembler.transform(unlabeled_data)

In [62]:
final_unlab.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [65]:
# Prepare the test data and transform 
test = final_unlab.select('features')
new_test_res = model.transform(final_unlab)

In [66]:
new_test_res.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.30451509165892...|[0.90925028722713...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.9586216743678...|[9.49503227406251...|       

In [67]:
new_test_res.select(['Company','prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

